# 🤖 Using Commercial LLMs Natively (Without Frameworks)

## Overview

In this notebook, we'll learn how to interact with popular **Commercial Large Language Model (LLM) APIs** directly using their official Python SDKs — **without any abstraction layers like LangChain**.

### Why Learn Native API Access?

Understanding native API access is crucial because:
1. **Foundation Knowledge**: It helps you understand what happens "under the hood" when using higher-level frameworks
2. **Debugging**: When things go wrong, knowing the native API helps you troubleshoot effectively
3. **Flexibility**: Some advanced features may only be available through native APIs
4. **Performance**: Direct API calls can sometimes be more efficient for simple use cases

### LLMs Covered in This Notebook

| Provider | Model | Pricing |
|----------|-------|---------|
| OpenAI | GPT-4o, GPT-4o-mini, GPT-3.5-Turbo | Paid (Pay-per-token) |
| Google | Gemini 2.5 Flash, Gemini 2.5 Pro | Free tier available |

### Prerequisites

- Basic Python knowledge
- API keys from OpenAI and/or Google
- Understanding of what LLMs are and their basic capabilities

In [22]:
# ============================================================================
# REQUIRED PACKAGES INSTALLATION
# ============================================================================
# Uncomment the lines below to install the required packages
# 
# openai: Official OpenAI Python client library for GPT models
# google-generativeai: Google's Python SDK for Gemini models
#
# The -qq flag suppresses output for cleaner installation
# ============================================================================

# !pip install -qq openai==1.57.0
# !pip install -qq google-generativeai==0.8.3

---

# 🔑 Part 1: OpenAI GPT Setup

## Step 1: Get Your OpenAI API Key

> ⚠️ **Important**: OpenAI API is a **paid service**. You'll need to set up billing before you can use it.

### What is an API Key?

An API key is like a password that:
- Identifies you to the OpenAI service
- Allows OpenAI to track your usage and bill you accordingly
- Should be kept **secret** and never shared publicly

### How to Get Your API Key

**Step 1.1**: Create an OpenAI account at [platform.openai.com](https://platform.openai.com)

**Step 1.2**: Set up billing
- Go to [Settings → Billing](https://platform.openai.com/settings/organization/billing/overview)
- Add a payment method and top up with at least $5 (sufficient for thousands of API calls)

![Billing Setup](https://i.imgur.com/pXgs31r.png)

**Step 1.3**: Generate your API key
- Navigate to [Dashboard → API Keys](https://platform.openai.com/api-keys)
- Click "Create new secret key"

![Create API Key](https://i.imgur.com/YbIBBtc.png)

**Step 1.4**: Save your key securely
- ⚠️ The key is shown **only once** — copy it immediately!
- Store it in a secure location (we'll use a `.env` file)

![Save API Key](https://i.imgur.com/myFXgZg.png)

> 💡 **Best Practice**: Never hardcode API keys in your code. Use environment variables or `.env` files instead.

## Step 2: Load OpenAI API Credentials

There are two common ways to load API credentials:

1. **Interactive Input** (using `getpass`) - Good for notebooks, prompts user at runtime
2. **Environment Variables** (using `.env` files) - Better for production, more secure

We'll demonstrate both approaches below.

In [23]:
# ============================================================================
# METHOD 1: Interactive Input (Alternative approach)
# ============================================================================
# getpass() prompts for input without showing what you type (like a password)
# This is useful when you don't want to store keys in files
# Uncomment below to use this method
# ============================================================================

# from getpass import getpass
# openai_key = getpass("Enter your OpenAI API Key: ")

In [24]:
# ============================================================================
# METHOD 2: Environment Variables (Recommended for production)
# ============================================================================
# python-dotenv loads variables from a .env file into environment variables
# 
# Your .env file should look like this:
# OPENAI_API_KEY=sk-your-api-key-here
# GOOGLE_GENAI_API_KEY=your-gemini-key-here
#
# Note: Add .env to your .gitignore to prevent accidental commits!
# ============================================================================

from dotenv import load_dotenv
import os

# load_dotenv() searches for .env file in current directory and parent directories
# Returns True if file found and loaded, False otherwise
load_dotenv()

# os.getenv() retrieves the value of an environment variable
# Returns None if the variable doesn't exist
openai_key = os.getenv("OPENAI_API_KEY")

# Optional: Verify the key was loaded (without printing the actual key!)
if openai_key:
    print("✅ OpenAI API key loaded successfully!")
else:
    print("❌ Warning: OpenAI API key not found. Please check your .env file.")

✅ OpenAI API key loaded successfully!


## Step 3: Using ChatGPT Directly via API

Now let's use the OpenAI API directly. This approach gives you:
- **Full control** over API parameters
- **Direct access** to all model features
- **Better understanding** of how chat completions work

### Key Concepts

| Concept | Description |
|---------|-------------|
| **Chat Completion** | The API endpoint for conversational AI (most common) |
| **Messages** | A list of message objects with roles (system, user, assistant) |
| **System Message** | Sets the behavior/personality of the AI |
| **User Message** | The actual prompt/question from the user |
| **Temperature** | Controls randomness (0 = deterministic, 2 = very random) |

In [25]:
# ============================================================================
# IMPORT THE OPENAI LIBRARY
# ============================================================================
# The openai library provides a simple interface to interact with OpenAI's APIs
# It handles HTTP requests, authentication, and response parsing for you
# ============================================================================

import openai

In [26]:
# ============================================================================
# CONFIGURE OPENAI WITH YOUR API KEY
# ============================================================================
# This sets the API key globally for all subsequent OpenAI API calls
# The library will automatically include this key in request headers
# ============================================================================

openai.api_key = openai_key

### Understanding API Pricing

OpenAI charges based on **tokens** — pieces of words (roughly 4 characters = 1 token).

#### What are Tokens?
- "Hello, world!" ≈ 4 tokens
- 1000 tokens ≈ 750 words
- You pay for both **input** (your prompt) and **output** (AI response) tokens

#### Model Recommendations

| Use Case | Recommended Model | Why |
|----------|------------------|-----|
| Cost-effective | GPT-4o-mini | Best price-to-performance ratio |
| High performance | GPT-4o | Most capable, best for complex tasks |
| Legacy/Budget | GPT-3.5-Turbo | Stable, cheaper, good for simple tasks |

📊 [View current pricing here](https://openai.com/api/pricing/)

![OpenAI Pricing](https://i.imgur.com/U0C1Xhx.png)

> 💡 **Tip**: Start with GPT-4o-mini for learning. Upgrade to GPT-4o only when you need superior reasoning.

### Your First API Call: Chat Completion

Let's make our first API call! We'll use the **Chat Completions** endpoint, which is designed for conversational interactions.

#### Understanding the Message Structure

```python
messages = [
    {"role": "system", "content": "..."},  # Sets AI behavior (optional but recommended)
    {"role": "user", "content": "..."},    # Your question/prompt
    {"role": "assistant", "content": "..."} # AI's previous response (for multi-turn)
]
```

In [27]:
# ============================================================================
# MAKING A CHAT COMPLETION REQUEST
# ============================================================================

# Step 1: Define the conversation as a list of messages
# Each message has a "role" and "content"
messages = [
    {
        "role": "system",           # System messages set the AI's behavior/personality
        "content": "You are a helpful assistant."
    },
    {
        "role": "user",             # User messages are your prompts/questions
        "content": "Who won the world series in 2020?"
    }
]

# Step 2: Make the API call
response = openai.chat.completions.create(
    model="gpt-4o-mini",    # The model to use (affects cost and capability)
    messages=messages,       # The conversation history
    temperature=0            # 0 = deterministic (same input → same output)
                            # Higher values (0.7-1.0) = more creative/random
)

# Step 3: Extract the response
# The response object contains metadata + the actual reply
# - response.choices: List of possible completions (usually just 1)
# - response.choices[0].message.content: The actual text response
print(response.choices[0].message.content)

The Los Angeles Dodgers won the World Series in 2020. They defeated the Tampa Bay Rays, clinching the championship in six games. This victory marked the Dodgers' first World Series title since 1988.


In [28]:
# ============================================================================
# CREATING A REUSABLE HELPER FUNCTION
# ============================================================================
# It's good practice to wrap API calls in functions for:
# 1. Reusability - call the same logic with different prompts
# 2. Abstraction - hide complexity from the rest of your code
# 3. Error handling - add try/except in one place (shown below)
# 4. Customization - easily switch models or parameters
# ============================================================================

def get_completion_chatgpt(prompt, model="gpt-4o-mini"):
    """
    Get a completion from OpenAI's ChatGPT model.
    
    This is a simplified wrapper that handles the boilerplate of making
    API calls, allowing you to focus on crafting good prompts.

    Args:
        prompt (str): The user prompt to send to the model.
        model (str): The model to use. Options include:
                    - "gpt-4o-mini" (default, cost-effective)
                    - "gpt-4o" (most capable)
                    - "gpt-3.5-turbo" (legacy, cheapest)

    Returns:
        str: The content of the response message.
        
    Example:
        >>> response = get_completion_chatgpt("What is 2+2?")
        >>> print(response)
        "2 + 2 equals 4."
    """
    # Construct the messages array with a default system message
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    
    # Make the API call with temperature=0 for consistent outputs
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0  # Deterministic output - good for testing and demos
    )
    
    # Return just the text content (not the full response object)
    return response.choices[0].message.content

In [29]:
# ============================================================================
# EXAMPLE 1: Using GPT-4o-mini (Cost-effective)
# ============================================================================
# GPT-4o-mini offers excellent performance at a fraction of the cost
# Perfect for most use cases including learning, prototyping, and production
# ============================================================================

prompt = 'Explain Generative AI in 2 bullet points'

# Call our helper function with the default model (gpt-4o-mini)
response = get_completion_chatgpt(prompt=prompt, model="gpt-4o-mini")

print("🤖 GPT-4o-mini Response:")
print("-" * 50)
print(response)

🤖 GPT-4o-mini Response:
--------------------------------------------------
- **Definition and Functionality**: Generative AI refers to a class of artificial intelligence models that can create new content, such as text, images, music, or videos, by learning patterns and structures from existing data. These models, like GPT-3 for text or DALL-E for images, use deep learning techniques to generate outputs that resemble human-created content.

- **Applications and Impact**: Generative AI has a wide range of applications, including content creation, design, gaming, and personalized experiences. It can enhance creativity, automate tasks, and provide innovative solutions across various industries, but it also raises ethical concerns regarding copyright, misinformation, and the potential for misuse.


In [30]:
# ============================================================================
# EXAMPLE 2: Using GPT-3.5-Turbo (Legacy/Budget option)
# ============================================================================
# GPT-3.5-Turbo is an older model but still capable and cheaper
# Notice how the response quality may differ from GPT-4o-mini
# This demonstrates how model choice affects output quality
# ============================================================================

prompt = 'Explain Generative AI in 2 bullet points'

# Same prompt, different model - compare the outputs!
response = get_completion_chatgpt(prompt=prompt, model="gpt-3.5-turbo")

print("🤖 GPT-3.5-Turbo Response:")
print("-" * 50)
print(response)

# 💡 Exercise: Compare the responses from both models. 
# Which one is more detailed? More accurate? Better structured?

🤖 GPT-3.5-Turbo Response:
--------------------------------------------------
- Generative AI refers to a type of artificial intelligence that is capable of creating new content, such as images, text, or music, based on patterns and data it has been trained on.
- It works by learning the underlying structure of the data it is trained on and then generating new content that is similar to the input data but not an exact copy, allowing for the creation of original and diverse outputs.


---

# 🔑 Part 2: Google Gemini Setup

## Step 1: Get Your Google Gemini API Key

> ✅ **Good News**: Google Gemini offers a **free tier** — no billing required for basic usage!

### Why Use Gemini?

- **Free tier available** — great for learning and experimentation
- **Multimodal capabilities** — can process text, images, audio, and video
- **Competitive performance** — Gemini 2.5 models rival GPT-4o in many benchmarks
- **Integration with Google ecosystem** — works well with other Google services

### How to Get Your API Key

**Step 1.1**: Go to [Google AI Studio](https://aistudio.google.com/app/u/0/apikey)
- Sign in with your Google account (Gmail works fine)

**Step 1.2**: Create your API key
- Click "Create API Key"
- Select or create a Google Cloud project

![Create Gemini API Key](https://i.imgur.com/UYVkKmK.png)

**Step 1.3**: Save your key securely
- Copy the key and store it in your `.env` file

![Save Gemini Key](https://i.imgur.com/9JZyw2t.png)

> 💡 **Note**: Check [Google's pricing page](https://ai.google.dev/pricing) for current free tier limits and paid options.

## Step 2: Load Gemini API Credentials

Similar to OpenAI, we'll load the Gemini API key from our `.env` file.

> 📝 **Your `.env` file should now contain:**
> ```
> OPENAI_API_KEY=sk-your-openai-key-here
> GOOGLE_GENAI_API_KEY=your-gemini-key-here
> ```

In [31]:
# ============================================================================
# LOAD GEMINI API KEY FROM ENVIRONMENT
# ============================================================================
# Same approach as OpenAI - load from .env file for security
# ============================================================================

from dotenv import load_dotenv
import os

# Reload .env file (in case it was updated after the OpenAI section)
load_dotenv()

# Get the Gemini API key from environment variables
gemini_key = os.getenv("GOOGLE_GENAI_API_KEY")

# Verify the key was loaded
if gemini_key:
    print("✅ Gemini API key loaded successfully!")
else:
    print("❌ Warning: Gemini API key not found. Please check your .env file.")

✅ Gemini API key loaded successfully!


## Step 3: Configure and Import the Gemini SDK

Now let's import Google's Generative AI library and configure it with our API key.

In [32]:
# ============================================================================
# IMPORT GOOGLE'S GENERATIVE AI LIBRARY
# ============================================================================
# google-generativeai is the official Python SDK for Google's Gemini models
# We import it as 'genai' by convention (shorter to type)
# 
# Note: You may see a TqdmWarning - this is just about progress bar display
# and doesn't affect functionality
# ============================================================================

import google.generativeai as genai

In [33]:
# ============================================================================
# CONFIGURE THE GEMINI SDK WITH YOUR API KEY
# ============================================================================
# Unlike OpenAI, Gemini uses a configure() function to set the API key
# This sets the key globally for all subsequent Gemini API calls
# ============================================================================

genai.configure(api_key=gemini_key)

print("✅ Gemini SDK configured successfully!")

✅ Gemini SDK configured successfully!


## Understanding Gemini Pricing & Models

Google offers generous free tiers for Gemini, making it excellent for learning!

### Gemini Model Recommendations

| Model | Best For | Free Tier |
|-------|----------|-----------|
| Gemini 2.5 Flash | Fast responses, cost-effective | ✅ Yes |
| Gemini 2.5 Pro | Complex reasoning, high quality | ✅ Limited |
| Gemini 2.0 Flash | Balanced speed/quality | ✅ Yes |

### Key Differences from OpenAI

| Feature | OpenAI | Gemini |
|---------|--------|--------|
| Free Tier | ❌ No | ✅ Yes |
| Context Window | Up to 128K tokens | Up to 1M tokens |
| Multimodal | Yes (text, images) | Yes (text, images, audio, video) |
| Native SDK | `openai` | `google-generativeai` |

📊 [View current Gemini pricing](https://ai.google.dev/pricing)

![Gemini Pricing](https://i.imgur.com/8hR2Ti8.png)

In [34]:
# ============================================================================
# LIST AVAILABLE GEMINI MODELS
# ============================================================================
# One advantage of the Gemini SDK is that you can programmatically list
# all available models and their capabilities
#
# Common supported methods:
# - 'generateContent' - Can generate text responses (what we need)
# - 'countTokens' - Can count tokens in a prompt
# - 'createCachedContent' - Supports content caching for efficiency
# - 'batchGenerateContent' - Can process multiple prompts at once
# ============================================================================

print("📋 Available Gemini Models with Content Generation:")
print("=" * 70)

for model in genai.list_models():
    # Filter to only show models that can generate content
    if 'generateContent' in model.supported_generation_methods:
        print(f"{model.name} -----> {model.supported_generation_methods}")

📋 Available Gemini Models with Content Generation:
models/gemini-2.5-flash -----> ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.5-pro -----> ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-exp -----> ['generateContent', 'countTokens', 'bidiGenerateContent']
models/gemini-2.0-flash -----> ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-001 -----> ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-exp-image-generation -----> ['generateContent', 'countTokens', 'bidiGenerateContent']
models/gemini-2.0-flash-lite-001 -----> ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-lite -----> ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-lite-preview-02-05 -----> ['

In [35]:
# ============================================================================
# YOUR FIRST GEMINI API CALL
# ============================================================================
# The Gemini API has a different structure than OpenAI:
# 1. Create a GenerativeModel instance with your model choice
# 2. Call generate_content() with your prompt
# 3. Access the text via response.text
#
# Note: Unlike OpenAI, you don't need to structure messages with roles
# for simple single-turn queries (though you can for multi-turn chats)
# ============================================================================

# Define your prompt
prompt = "What is the capital of France?"

# Step 1: Create a model instance
# GenerativeModel is a class that wraps a specific Gemini model
model = genai.GenerativeModel("gemini-2.5-flash")

# Step 2: Generate content
# This makes the actual API call to Google's servers
response = model.generate_content(prompt)

# Step 3: Access the response text
# The response object contains metadata and the actual text reply
print("🤖 Gemini Response:")
print("-" * 50)
print(response.text)

🤖 Gemini Response:
--------------------------------------------------
The capital of France is **Paris**.


In [36]:
# ============================================================================
# CREATING A REUSABLE HELPER FUNCTION FOR GEMINI
# ============================================================================
# Just like we did for OpenAI, let's create a simple wrapper function
# This makes it easy to switch between models and reuse code
# ============================================================================

def get_completion_gemini(prompt, model_name="gemini-2.5-flash"):
    """
    Get a completion from Google's Gemini model.
    
    This is a simplified wrapper that handles the boilerplate of making
    API calls to Google's Gemini service.

    Args:
        prompt (str): The user prompt to send to the model.
        model_name (str): The name of the Gemini model to use. Options include:
                         - "gemini-2.5-flash" (default, fast and capable)
                         - "gemini-2.5-pro" (most capable)
                         - "gemini-2.0-flash" (balanced)

    Returns:
        str: The text of the response.
        
    Example:
        >>> response = get_completion_gemini("What is 2+2?")
        >>> print(response)
        "2 + 2 equals 4."
    """
    # Create a model instance for the specified model
    model = genai.GenerativeModel(model_name)
    
    # Generate content and return the text response
    response = model.generate_content(prompt)
    return response.text

print("✅ Helper function 'get_completion_gemini' defined successfully!")

✅ Helper function 'get_completion_gemini' defined successfully!


In [37]:
# ============================================================================
# TESTING THE GEMINI HELPER FUNCTION
# ============================================================================
# Let's use the same prompt we used with OpenAI to compare responses
# This is a great way to evaluate different models for your use case
# ============================================================================

prompt = 'Explain Generative AI in 2 bullet points'

# Use our helper function to get a response from Gemini 2.5 Flash
response = get_completion_gemini(prompt=prompt, model_name="gemini-2.5-flash")

print("🤖 Gemini 2.5 Flash Response:")
print("-" * 50)
print(response)

# 💡 Exercise: Compare this response to the GPT-4o-mini and GPT-3.5-Turbo
# responses from earlier. Notice any differences in style, detail, or accuracy?

🤖 Gemini 2.5 Flash Response:
--------------------------------------------------
Here's an explanation of Generative AI in two bullet points:

*   **Creates novel content:** Unlike traditional AI that analyzes or classifies existing data, Generative AI models are designed to produce entirely new and original content, such as text, images, audio, or video, that didn't exist before.
*   **Learns patterns from vast datasets:** These models are trained on enormous amounts of existing data to understand the underlying patterns, structures, and styles, which they then use to generate new outputs that are coherent and often indistinguishable from human-created content.


# 📝 Summary & Key Takeaways

## What We Learned

In this notebook, we explored how to use commercial LLM APIs **natively** (without frameworks like LangChain):

### OpenAI GPT
- Uses the `openai` Python library
- Requires a **paid API key** with billing set up
- Uses a **messages array** structure with roles (system, user, assistant)
- Main method: `openai.chat.completions.create()`

### Google Gemini
- Uses the `google-generativeai` Python library
- Offers a **free tier** for learning and experimentation
- Simpler API structure with `GenerativeModel` class
- Main method: `model.generate_content()`

## Comparison Quick Reference

| Feature | OpenAI | Gemini |
|---------|--------|--------|
| Import | `import openai` | `import google.generativeai as genai` |
| Configure | `openai.api_key = key` | `genai.configure(api_key=key)` |
| API Call | `openai.chat.completions.create()` | `model.generate_content()` |
| Response | `response.choices[0].message.content` | `response.text` |

## Next Steps

1. **Practice**: Try different prompts with both APIs
2. **Experiment**: Change the `temperature` parameter and observe how outputs vary
3. **Compare**: Test the same prompt across different models
4. **Move On**: Learn how to use these same LLMs with **LangChain** in the next notebook!

> 🎯 **Coming Up Next**: In `2. Commercial_LLMs_with_LangChain.ipynb`, we'll see how LangChain simplifies working with these APIs and provides powerful abstractions for building LLM applications.
